In [1]:
####MAKE API CALL TO KAGGLE TO GET THE DATASET
!pip install kaggle

     -------------------------------------- 82.1/82.1 kB 921.2 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kaggle: filename=kaggle-1.6.14-py3-none-any.whl size=105134 sha256=0f0f6e71786d347c85d635969f8b4219898cd5bc0a73bf0d094eb41f8cb48131
  Stored in directory: c:\users\arnab\appdata\local\pip\cache\wheels\fc\09\1c\4fee6854faf2dc17e39eb1fe79acd7f441378b0f58108b88c3
Successfully built kaggle


In [2]:
import kaggle

In [3]:
!kaggle datasets download shivamb/netflix-shows -f netflix_titles.csv

Dataset URL: https://www.kaggle.com/datasets/shivamb/netflix-shows
License(s): CC0-1.0




  0%|          | 0.00/1.34M [00:00<?, ?B/s]
 75%|#######4  | 1.00M/1.34M [00:01<00:00, 680kB/s]
100%|##########| 1.34M/1.34M [00:01<00:00, 853kB/s]


In [6]:
####EXTRACT THE CSV FROM ZIP FILE
import zipfile

# Specify the path to the zip file and the name of the csv file within the zip
zip_file_path = 'netflix_titles.csv.zip'
csv_file_name = 'netflix_titles.csv'

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract the specified CSV file
    zip_ref.extract(csv_file_name, '.')

print(f"Extracted {csv_file_name} from {zip_file_path}")

Extracted netflix_titles.csv from netflix_titles.csv.zip


In [7]:
####READ DATA FROM THE CSV
import pandas as pd
df_netflix=pd.read_csv('netflix_titles.csv')
df_netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [8]:
####LOAD TO SQL Server
import sqlalchemy as sal
engine=sal.create_engine('mssql://LAPTOP-R3E2GC1C\SQLEXPRESS01/master?driver=ODBC+Driver+17+for+SQL+Server')
conn=engine.connect()

In [11]:
###LOAD IT TO SQL AND GET THE TABLE SCHEMA 
df_netflix.to_sql('netflix_raw',con=conn,index=False,if_exists='replace')

-1

In [15]:
###Title has foreign characters. In SQL change the datatype to nvarchar to be able to handle multiple languages
cond=(df_netflix['show_id']=='s5023')
df_netflix.loc[cond,'title']

5022    반드시 잡는다
Name: title, dtype: object

In [18]:
###Find the maximum length of item in each column
max(df_netflix.show_id.str.len())

5

In [29]:
for col in df_netflix.columns:
    print(f'column {col}:')
    if col!= 'release_year':print(max(df_netflix[col].dropna().str.len()))

column show_id:
5
column type:
7
column title:
104
column director:
208
column cast:
771
column country:
123
column date_added:
19
column release_year:
column rating:
8
column duration:
10
column listed_in:
79
column description:
248


In [30]:
###LOAD IT TO SQL USING THE NEW TABLE SCHEMA CREATED ON SQL
df_netflix.to_sql('netflix_raw',con=conn,index=False,if_exists='append')

-1

In [31]:
conn.close()

In [32]:
###Find columns with null values
df_netflix.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64